In [11]:
from PIL import Image
import numpy as np 
import matplotlib.pyplot as plt
import cv2
import torch
import pandas as pd


In [12]:
detect_model = torch.hub.load('../yolov5/', 'custom', path='../weights/detect_weights.pt', force_reload=True, source='local')

YOLOv5 🚀 v7.0-212-g9974d51 Python-3.8.10 torch-2.0.1+cu117 CPU

Fusing layers... 
Model summary: 157 layers, 7020913 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [13]:
def red_color_seg(image):

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # lower boundary RED color range values; Hue (0 - 10)
    lower1 = np.array([0, 120, 0])
    upper1 = np.array([12, 255, 255])

    # upper boundary RED color range values; Hue (160 - 180)
    lower2 = np.array([160,120,0])
    upper2 = np.array([185,255,255])

    lower_mask = cv2.inRange(hsv_image, lower1, upper1)
    upper_mask = cv2.inRange(hsv_image, lower2, upper2)
    red_mask = lower_mask | upper_mask

    red_seg = cv2.bitwise_and(image, image, mask=red_mask)
    cv2.imwrite("stage1.jpg", red_seg)


    # dont change 5
    kernel = np.ones((5, 5), np.uint8)
    red_mask = cv2.dilate(red_mask, kernel, iterations=1)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_OPEN, kernel)
    red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel)

    # kernel = np.ones((7, 7), np.uint8)

    red_segmented_image = cv2.bitwise_and(image, image, mask=red_mask)
    # cv2.imwrite("stage2.jpg", red_segmented_image)
    # red_segmented_image = cv2.fastNlMeansDenoisingColored(red_segmented_image, None,30,30,15,30) 

    gray = cv2.cvtColor(red_segmented_image, cv2.COLOR_BGR2GRAY)

    # Find contours in the opened maskqqqqqqqqq
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    min_area_threshold = 0
    max_area_threshold = 8000
    # Filter out contours with area less than the threshold and fill them with black color
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_area_threshold or area > max_area_threshold:
            cv2.drawContours(gray, [contour], 0, (0, 255, 0), -1)



    # kernel = np.ones((3, 3), np.uint8)
    # red_mask = cv2.erode(red_mask, kernel, iterations=1)
    red_segmented_image = cv2.bitwise_and(image, image, mask=gray)
    # cv2.imwrite("stage3.jpg", red_segmented_image)
    # print(red_segmented_image.shape)

    

    # red_segmented_image = cv2.GaussianBlur(red_segmented_image, (7, 7), 0)
    # red_segmented_image = cv2.fastNlMeansDenoisingColored(red_segmented_image, None,30,30,10,30) 
    # red_segmented_image = cv2.GaussianBlur(red_segmented_image, (5, 5), 0)
    # image = cv2.addWeighted(image, 1, red_segmented_image, 0.5, 0)

    return red_segmented_image, gray
    # cv2.imshow("out", red_segmented_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [30]:
def denoise(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Reshape the image to a 2D array of pixels
    pixel_values = lab_image.reshape((-1, 3))

    # Remove black pixels (where L = 0)
    non_black_pixels = pixel_values[pixel_values[:, 0] != 0]

    # Perform k-means clustering
    k = 5  # Number of clusters
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(np.float32(non_black_pixels), k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Initialize a mask to store clusters with enough pixels
    mask = np.zeros_like(labels)

    # Count the number of pixels in each cluster
    for i in range(k):
        mask[labels.ravel() == i] = np.count_nonzero(labels == i)

    # Filter out clusters with fewer than a specified number of pixels
    threshold = 100  # Minimum number of pixels per cluster
    mask[mask < threshold] = 0

    # Create a binary mask for the filtered clusters
    filtered_mask = (mask > 0).astype(np.uint8)

    # Reshape the mask to the original image shape
    filtered_mask = filtered_mask.reshape(lab_image.shape[:2])

    # Apply the filtered mask to the original image
    result = cv2.bitwise_and(image, image, mask=filtered_mask)

    # Display the result
    return result


In [3]:
def circle_seg(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=0.1, minDist=50, param1=100, param2=40, minRadius=10, maxRadius=100)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for (x, y, r) in circles:
            cv2.circle(image, (x, y), r, (0, 255, 0), 4)

    return image

In [4]:
def tria_seg(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect edges using Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)




    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through contours and approximate them to identify triangles
    for contour in contours:
        # Approximate the contour to a polygon
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.03 * perimeter, True)

        # If the approximated polygon has 3 vertices, it's a triangle
        if len(approx) == 3:
            cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)

    return image

In [22]:
video_capture = cv2.VideoCapture('/home/ashmitha/traffic_sign_detection/test_images/sunlight_straight.mkv')

# frame_rate = video_capture.get(cv2.CAP_PROP_FPS)
# print(frame_rate)

# # Calculate the new frame rate (double the original frame rate)
# new_frame_rate = frame_rate * 10

# # Set the new frame rate for the video capture
# video_capture.set(cv2.CAP_PROP_FPS, new_frame_rate)
# frame_rate = video_capture.get(cv2.CAP_PROP_FPS)
# print(frame_rate)

detection_data = []



In [41]:
cv2.destroyAllWindows()


In [1]:
detected_correct = 0
detected_wrong = 0
did_not_detect = 0

In [57]:
while True:
    detected_correct += int(input("Enter the number of true positives: "))
    


In [15]:
classes = ['prohibitory', 'danger', 'mandatory', 'other']

In [19]:
while True:
    # Read the next frame
    ret, frame = video_capture.read()
    # cv2.imshow("out", frame)

    frame_copy = frame.copy()
    
    seg_frame, mask = red_color_seg(frame)

    results = detect_model(frame)

    bounding_boxes = results.xyxy[0].cpu().numpy()
    cv2.putText(frame_copy, str(video_capture.get(cv2.CAP_PROP_POS_MSEC)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)

    
    for box in bounding_boxes:
        x_min, y_min, x_max, y_max, confidence, class_id = box

        if not mask[int(y_min):int(y_max), int(x_min):int(x_max)].any():
            confidence = confidence * 0.5

        if  confidence > 0.4:
            cv2.putText(frame_copy, str(confidence), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
            cv2.rectangle(frame_copy, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
            class_ = classes[int(class_id)]
            detection_data.append([video_capture.get(cv2.CAP_PROP_POS_MSEC), class_, confidence])

    # for box in bounding_boxes:
    #     x_min, y_min, x_max, y_max, confidence, class_id = box
    #     if confidence > 0.4:
    #         cv2.rectangle(frame_copy, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)



    concatenated_frame = cv2.hconcat([frame, seg_frame])


    # Display the concatenated frame
    cv2.imshow("Original vs Processed", frame_copy)
    

    # # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    if cv2.waitKey(0) & 0xFF == ord('n'):
        continue

    if cv2.waitKey(0) & 0xFF == ord('s'):
        detection_data.append(["start frame",video_capture.get(cv2.CAP_PROP_POS_MSEC), ])

    if cv2.waitKey(0) & 0xFF == ord('e'):
        detection_data.append(["end frame",video_capture.get(cv2.CAP_PROP_POS_MSEC), ])
    
    ret, frame = video_capture.read()
   


# Release the video capture object and close the display window
video_capture.release()
cv2.destroyAllWindows()

In [20]:
df = pd.DataFrame(detection_data, columns=['Timestamp', 'ClassID', 'Confidence'])

# Write the DataFrame to a CSV file
df.to_csv('dark_sign.csv', index=False)

In [33]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data from the CSV file
df = pd.read_csv('plot.csv', header=None)

# Extract the first and last numerical values
first_value = df.iloc[:,0]
last_value = df.iloc[:,2]



In [37]:
type(first_value)

pandas.core.series.Series

In [21]:
import matplotlib.pyplot as plt

data = [
    (96468.0, 'x', 0.0),
    (98001.0, 'prohibitory', 0.4190042316913605),
    (98334.0, 'danger', 0.4456574022769928),
    (99468.0, 'mandatory', 0.5257366299629211),
    (99534.0, 'mandatory', 0.6467638611793518),
    (99601.0, 'mandatory', 0.43351343274116516),
    (99668.0, 'mandatory', 0.7146356701850891),
    (99734.00000000001, 'mandatory', 0.5248815417289734),
    (99801.0, 'mandatory', 0.5179475545883179),
    (99801.0, 'prohibitory', 0.46880823373794556),
    (99868.0, 'prohibitory', 0.4657275676727295),
    (99934.0, 'mandatory', 0.47248154878616333),
    (100068.0, 'prohibitory', 0.5615005493164062),
    (100201.00000000001, 'prohibitory', 0.616981565952301),
    (100268.0, 'other', 0.468959242105484),
    (100334.0, 'other', 0.5272660255432129),
    (100401.0, 'prohibitory', 0.8893968462944031),
    (100468.0, 'other', 0.4021945893764496),
    (100601.0, 'other', 0.563284695148468),
    (100668.0, 'other', 0.5965499877929688),
    (100934.0, 'other', 0.44282960891723633),
    (101068.0, 'prohibitory', 0.7431638240814209)
]



# Extract x and y values
x_values = [item[0] for item in data]
y_values = [item[2] for item in data]

# Plot the arrays against each other
plt.figure(figsize=(8, 6))
plt.plot(x_values, y_values, marker='o', linestyle='-')
plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.title('Plot of Values against Timestamp')
plt.grid(True)
plt.show()


In [39]:
# Plot the value
# plt.figure(figsize=(8, 6))
plt.plot([np.array(first_value), np.array(last_value)], marker='o', color='blue')
plt.xlabel('First Value')
plt.ylabel('Last Value')
plt.title('Plot of First vs Last Numerical Value')
# plt.grid(True)
plt.show()

In [110]:
img = cv2.imread("/home/ashmitha/traffic_sign_detection/test_images/test1.jpg")

img = red_color_seg(img)
img = circle_seg(img)
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [66]:
img = cv2.imread("/home/ashmitha/traffic_sign_detection/scripts/x.jpg")
cv2.imshow("Image", img)

In [94]:
cv2.destroyAllWindows()


In [103]:
import cv2
import numpy as np

# Read the input image
image = cv2.imread("../scripts/x.jpg")  # Replace "triangle_image.jpg" with the path to your image

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Detect edges using Canny edge detection
edges = cv2.Canny(blurred, 50, 150)




# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Iterate through contours and approximate them to identify triangles
for contour in contours:
    # Approximate the contour to a polygon
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.03 * perimeter, True)

    print(len(approx))
    # If the approximated polygon has 3 vertices, it's a triangle
    if len(approx) == 8:
        print("reached")
        cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)

# # Display the result
cv2.imshow("Triangles Detected", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [7]:
import os

path = "../data/GTSDB/train_detection/images/"
test_images = os.listdir(path)

In [13]:
import time

In [31]:
image_path = "00298.jpg"
img = cv2.imread(path + image_path)
og_img = img.copy()

cv2.imshow("og", img)
cv2.waitKey(0)

cv2.destroyAllWindows()

In [36]:
img = red_color_seg(img)

# cir_img = circle_seg(img.copy())
# tria_img = tria_seg(img.copy())
# cv2.imwrite("circle.jpg", cir_img)
# cv2.imwrite("tria.jpg", tria_img)
# results_image = np.hstack((tria_img, cir_img, og_img))
# image_rgb = cv2.cvtColor(results_image, cv2.COLOR_BGR2RGB)
cv2.imshow("Side by Side", img)
# plt.show()
cv2.waitKey(0)

cv2.destroyAllWindows()

In [10]:
for image_path in test_images:
    image_path = "00298.jpg"
    img = cv2.imread(path + image_path)
    og_img = img.copy()

    start_time = time.time()
    img = red_color_seg(img)
    time = time.time() - start_time
    print(time)


    cir_img = circle_seg(img.copy())
    tria_img = tria_seg(img.copy())
    cv2.imwrite("circle.jpg", cir_img)
    cv2.imwrite("tria.jpg", tria_img)
    results_image = np.hstack((tria_img, cir_img, og_img))
    image_rgb = cv2.cvtColor(results_image, cv2.COLOR_BGR2RGB)
    cv2.imshow("Side by Side", image_rgb)
    # plt.show()
    cv2.waitKey(0)

    cv2.destroyAllWindows()
    break

0.0448603630065918
